In [1]:
%%capture
%load_ext autoreload
%autoreload 2

In [2]:
# region General Imports
import os
import uuid
import shutil
import time
import random
import datetime
import glob
import pickle
import tqdm
import copy
import optuna
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import seaborn as sns
import mne
from rich import print as rprint
from rich.pretty import pprint as rpprint
from tqdm import tqdm
from itertools import chain
from functools import partial
# endregion General Imports

from model_optim.model_optimizer import ModelOptimizer

# Dataset
from custom_datasets.fatigue_mi import FatigueMI
from custom_datasets.norm_cho import NormCho2017
from custom_datasets.opt_game_mi import OptGameMI
from custom_datasets.opt_std_mi import OptStdMI

2024-04-12 21:53:09.723617: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-12 21:53:09.723662: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-12 21:53:09.724862: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-12 21:53:09.732517: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-12 21:53:10.592405: W tensorflow/compiler/tf2

To use the get_shape_from_baseconcar, InputShapeSetterEEG, BraindecodeDatasetLoaderyou need to install `braindecode`.`pip install braindecode` or Please refer to `https://braindecode.org`.


<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
/home/arazzz/anaconda3/envs/moabb_model_optimization_quant/lib/python3.11/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


In [3]:
MODELS_LIST = [
    # "shallow_conv_net",
    "lstm_net",
    "deep_conv_net",
    "eeg_net",
    "lstm_cnn_net",
    "lstm_cnn_net_v2"
]
MODELS_HYPERPARAMS_DICT = {
    "shallow_conv_net": {
        "max_epochs": 5
    },
    "eeg_net": {
        "max_epochs": 10
    },
    "deep_conv_net": {
        "max_epochs": 10
    },
    "lstm_net": {
        "max_epochs": 10
    },
    "lstm_cnn_net": {
        "max_epochs": 10
    },
    "lstm_cnn_net_v2": {},
}

DATASETS_LIST = [
    FatigueMI,
    # NormCho2017,
    # OptGameMI,
    # OptStdMI,
]

In [4]:
# from model_optim.utils import data_generator

# data_generator(
#     dataset=NormCho2017(),
#     subjects=[1],
#     channel_idx=[],
#     sfreq=128,
# )

In [4]:
subject_files = glob.glob("./temp_v2/*/*/model/study_best_trial.npy")
subject_files_data = {}
for subject_file in subject_files:
    subject_files_data[subject_file] = np.load(subject_file, allow_pickle=True).item()


In [5]:
for subject_file_data_key, subject_file_data in subject_files_data.items():
    rprint(subject_file_data_key, subject_file_data.user_attrs['trial_data']['test_accuracy'])

./temp_v2/[9]/63a8c87ffc02471893db5ac9a0781946/model/study_best_trial.npy 0.9090909361839294

./temp_v2/[10]/c2cc69dca74d4bfa81722cd634e6403e/model/study_best_trial.npy 0.7727273106575012

./temp_v2/[11]/e0643f9a780146a4adc15ddd4a9ff053/model/study_best_trial.npy 0.7272727489471436

./temp_v2/[2]/6405d11e654b42aca9df48458c67ecde/model/study_best_trial.npy 0.6363636255264282

./temp_v2/[7]/aabe056cd1954a6f92ab47d84c86b1b8/model/study_best_trial.npy 0.7272727489471436

...

./temp_v2/[13]/1b189965ada44ff99e73fa145cd3901d/model/study_best_trial.npy 0.6363636255264282

./temp_v2/[5]/d199c9c2ac924b238693f158eb88f675/model/study_best_trial.npy 0.6363636255264282

./temp_v2/[12]/96dc576945fb4f2db582d66ae1d2c8ce/model/study_best_trial.npy 0.7727273106575012

./temp_v2/[8]/9fd82ec44ef3496da6307b57ecf4532f/model/study_best_trial.npy 0.6363636255264282

./temp_v2/[4]/3623cb4ba1ad4a908c9098f5297a6778/model/study_best_trial.npy 0.8636363744735718

In [12]:
for dataset in [FatigueMI]:
    for model in ["lstm_cnn_net"]:
        model_optimizer = ModelOptimizer(
            dataset=dataset(),
            model_name=model
        )
        for subject in [12]:
            max_epochs = MODELS_HYPERPARAMS_DICT[model]["max_epochs"]
            study = model_optimizer.search_best_model(
                subjects = [subject],
                max_iter = 25,
                max_epochs = max_epochs,
                max_stag_count = 10,
                rounds = 1,
                replace_previous_study_for_subjects = False
            )

Original sfreq: 300.0


  0%|          | 0/25 [00:00<?, ?it/s]

Using sfreq: 256; sfreq is None: True; self.original_sfreq: 300.0; sfreq=256
Adding metadata with 3 columns
Epoch 1/10
1/1 [==============================] - ETA: 0s - loss: 116.7297 - accuracy: 0.5735
Epoch 00001: val_accuracy improved from -inf to 0.61111, storing weights.
1/1 [==============================] - 3s 3s/step - loss: 116.7297 - accuracy: 0.5735 - val_loss: 105.7196 - val_accuracy: 0.6111 - lr: 0.0010
Epoch 2/10
1/1 [==============================] - ETA: 0s - loss: 105.7729 - accuracy: 0.5441
Epoch 00002: val_accuracy did not improve
1/1 [==============================] - 0s 44ms/step - loss: 105.7729 - accuracy: 0.5441 - val_loss: 98.4783 - val_accuracy: 0.5556 - lr: 0.0010
Epoch 3/10
1/1 [==============================] - ETA: 0s - loss: 98.6259 - accuracy: 0.5882
Epoch 00003: val_accuracy did not improve
1/1 [==============================] - 0s 46ms/step - loss: 98.6259 - accuracy: 0.5882 - val_loss: 92.7606 - val_accuracy: 0.6111 - lr: 0.0010
Epoch 4/10
1/1 [=======

In [13]:
model_optimizer.get_study_metrics(study).sort_values(by="scores", ascending=True)

,train_acc,test_acc,val_acc,train_val_acc_diff,train_loss,val_loss,train_val_loss_diff,test_loss,scores,channels_selected,sfreq,batch_size
12,1.000000,0.545455,0.666667,0.333333,16.501001,16.509830,0.008829,16.570940,0.111511,"[P3, Fz, Fp1, T3, O2, F7, F8, A2]",300.0,224
16,0.882353,0.545455,0.666667,0.215686,16.650978,16.384975,0.266003,18.711729,0.111611,"[P3, C3, Fz, Pz, Fp1, T3, O2, F7, F8, T6]",128.0,224
5,1.000000,0.636364,0.666667,0.333333,9.176478,9.411931,0.235453,9.479000,0.111611,"[P3, C3, Pz, Fp1, Fp2, T3, O1, O2, F8, T6]",300.0,224
14,1.000000,0.409091,0.666667,0.333333,18.201464,17.866186,0.335278,18.061745,0.111661,"[P3, Fz, P4, Pz, Fp1, T3, O1, O2, F7, F8, A2]",300.0,224
18,0.955882,0.454545,0.666667,0.289216,18.810242,18.693073,0.117168,23.505180,0.111711,"[P3, F3, P4, Cz, Pz, Fp1, Fp2, T3, T5, O2, F7,...",300.0,192
6,1.000000,0.454545,0.611111,0.388889,12.835961,13.050479,0.214518,14.696945,0.151535,"[Fz, Fp1, F7, F8, A2, T4]",256.0,192
11,0.985294,0.454545,0.611111,0.374183,5.690474,6.120966,0.430493,6.868915,0.151635,"[P3, C3, Fz, Fp1, T3, F7, F8, A2]",128.0,256
13,0.941176,0.409091,0.611111,0.330065,13.613918,13.692876,0.078958,15.394052,0.151785,"[P3, C3, Fz, Fp1, Fp2, T3, O2, F7, F8, A2, T6]",300.0,224
21,0.911765,0.500000,0.555556,0.356209,15.989138,15.926647,0.062490,20.406557,0.198031,"[P3, C3, Fz, Pz, Fp1, T3, O2, F7, F8, T6]",128.0,224
22,0.691176,0.500000,0.555556,0.135621,20.893789,20.521727,0.372063,27.163294,0.198031,"[P3, C3, Fz, Pz, Fp1, T3, O2, F7, F8, T6]",128.0,192


In [14]:
rpprint({ k: v for k, v in study.best_trial.params.items() if not k.startswith("channels") })
rprint("test_accuracy =", study.best_trial.user_attrs["trial_data"]["test_accuracy"])
rprint("val_accuracy =", np.max(study.best_trial.user_attrs["trial_data"]["val_accuracy"]))
rprint("channels_selected =", study.best_trial.user_attrs["trial_data"]["channels_selected"])

{
│   'sfreq': None,
│   'batch_size': 224,
│   'conv1d_1_units': 70,
│   'conv1d_1_kernel_size': 78,
│   'conv1d_1_strides': 3,
│   'conv1d_1_maxpool_strides': 1,
│   'lstm_1_units': 100,
│   'l2_reg_1': 0.3383681530636568,
│   'l2_reg_2': 0.16073380063395976,
│   'l2_reg_3': 0.23971068335483192,
│   'dropout_rate_1': 0.4,
│   'dropout_rate_2': 0.6
}

test_accuracy = 0.5454545617103577

val_accuracy = 0.6666666865348816

channels_selected = ['P3' 'Fz' 'Fp1' 'T3' 'O2' 'F7' 'F8' 'A2']